<a href="https://colab.research.google.com/github/svantepihl/2IS063-Project/blob/main/2IS063_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Your Data Mining Pilot Study and Plan




## Background

*Describe the organization and its operations, operational environment, analytical needs, etc.*

*Replace the contents of this cell with your own text.*

## Problem description

*Describe the problem that the organization is facing and the research question it needs answered.*

*Replace the contents of this cell with your own text.*

## Data collection
The dataset on which our analysis is based on comes from the online data science community [Kaggle](https://kaggle.com), a subsidiary of Google LLC.

The dataset was gathered and uploaded to Kaggle by the user Manch Hui, it can be found [here](https://www.kaggle.com/manchunhui/us-election-2020-tweets). The dataset was gathered by using the Twitter API and a scraping tool called 'snscrape'. As of writing (11 November 2020) the dataset consists of 1 727 000 tweets. All tweets in the dataset added beacause they contain at least one of the following hasgtags:

> #JoeBiden


> #DonaldTrump

We start with downloading the ZIP-file containing the latest version of the  dataset from kaggle.






In [1]:
# Import packages
import pandas as pd
import time
from IPython.display import clear_output
!pip install kaggle
!mkdir ~/.data 
!echo '{"username":"svantepihl","key":"294e719d5da4fffa993782c856e0a64d"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.data/kaggle.json  # set permissions
clear_output()

# Download data from Kaggle using Svante's API Key
!kaggle datasets download -d "manchunhui/us-election-2020-tweets" -p /content/data/
time.sleep(.3) # wait for file to be written
!unzip /content/data/us-election-2020-tweets.zip -d "/content/data/"

## Read into dataframes
tweets_trump = pd.read_csv('/content/data/hashtag_donaldtrump.csv', lineterminator='\n')
tweets_trump['hashtag'] = "trump"
tweets_biden = pd.read_csv('/content/data/hashtag_joebiden.csv', lineterminator='\n')
tweets_biden['hashtag'] = 'biden'
tweets_all = tweets_trump.append(tweets_biden, ignore_index=True)

Let's have a look at the columns aviable in our dataset:

In [ ]:
# Select only tweets from the US
tweets_biden = tweets_biden[tweets_biden.country == 'United States of America']
tweets_trump = tweets_trump[tweets_trump.country == 'United States of America']

# Remove all rows where location is missing
#tweets_biden = tweets_biden.dropna(subset=['lat','long','city','country','continent','state','state_code'], how='any')
#tweets_trump = tweets_trump.dropna(subset=['lat','long','city','country','continent','state','state_code'], how='any')

tweets_biden.info()
tweets_trump.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776886 entries, 0 to 776885
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   created_at            776886 non-null  object 
 1   tweet_id              776886 non-null  float64
 2   tweet                 776886 non-null  object 
 3   likes                 776886 non-null  float64
 4   retweet_count         776886 non-null  float64
 5   source                776173 non-null  object 
 6   user_id               776886 non-null  float64
 7   user_name             776868 non-null  object 
 8   user_screen_name      776886 non-null  object 
 9   user_description      694880 non-null  object 
 10  user_join_date        776886 non-null  object 
 11  user_followers_count  776886 non-null  float64
 12  user_location         543095 non-null  object 
 13  lat                   355293 non-null  float64
 14  long                  355293 non-null  float64
 15  

In [ ]:
# Encode in GeoDataFrame
!pip install geopandas
#!apt install libspatialindex-dev
#!pip install rtree
import geopandas
import matplotlib.pyplot as plt
from geopandas.tools import sjoin

gdf_biden = geopandas.GeoDataFrame(tweets_biden, geometry=geopandas.points_from_xy(tweets_biden.long, tweets_biden.lat))
gdf_biden = gdf_biden.dropna(subset=['geometry'])

gdf_trump = geopandas.GeoDataFrame(tweets_trump, geometry=geopandas.points_from_xy(tweets_trump.long, tweets_trump.lat))
gdf_trump = gdf_trump.dropna(subset=['geometry'])

gdf_biden.reset_index()
gdf_trump.reset_index()

gdf_biden.info()
gdf_trump.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 90642 entries, 6 to 776865
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   created_at            90642 non-null  object  
 1   tweet_id              90642 non-null  float64 
 2   tweet                 90642 non-null  object  
 3   likes                 90642 non-null  float64 
 4   retweet_count         90642 non-null  float64 
 5   source                90639 non-null  object  
 6   user_id               90642 non-null  float64 
 7   user_name             90642 non-null  object  
 8   user_screen_name      90642 non-null  object  
 9   user_description      85764 non-null  object  
 10  user_join_date        90642 non-null  object  
 11  user_followers_count  90642 non-null  float64 
 12  user_location         90642 non-null  object  
 13  lat                   90642 non-null  float64 
 14  long                  90642 non-null  float64

In [ ]:
# Get urban or rural
!pip install pyshp
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 30)
import shapefile
import fiona
from shapely.geometry import Point 
from shapely.geometry import shape
from shapely.geometry import Polygon, Point, MultiPolygon

points = gdf_biden.reset_index()


sf = shapefile.Reader("/gdrive/My Drive/Colab Notebooks/cb_2018_us_ua10_500k/cb_2018_us_ua10_500k")
shapes = sf.shapes()

polygons = [ Polygon(shape.points) for shape in shapes ]

def is_urban(pt):
    result = False
    count = sum(map(lambda x : x.contains(pt), polygons))
    if count > 0:
      result = True
    return result


points['urban'] = points['geometry'].apply(is_urban)
print(points.head(30))


## Data analysis

*Document you choice and motivation for selected data mining method(s) here. Choose a data mining method(s) to use in Python code to perform an analysis of your chosen dataset. Describe why you chose the method(s) and what interesting things you have found from the analysis.*

*Replace the contents of this cell with your own text.*

In [ ]:
# Add your own code

## Evaluation of results

*Document an evaluation your analysis results and describe how potentially actionable they are.*

*Replace the contents of this cell with your own text.*

In [ ]:
# Add your own code

## Schedule and description of project plan

*Rough schedule for the project beyond the pilot study presented in 3-5. This does not have to be advanced, you can simply provide an estimate based upon reported schedules for similar projects in the literature.*

*Replace the contents of this cell with your own text.*

## Ethical aspects that need to be considered

*Are there ethical aspects that need to be considered? Are there legal implications (e.g., personal data / GDPR)? Are there implications if the case organization is a business, public authority, or nonprofit entity?*

*Replace the contents of this cell with your own text.*